# Sensitivities and symbolic differentiation

The standard formulation of forward sensitivities is

 - State: $y = y(p, t)$
 - Constant or an initial state value: $p$ 
 - State derivatives: $f(y, p) = \frac{\mathrm{d}y}{\mathrm{d}t}$, this is the myokit.Model
 - Sensitivities: $s = \frac{\mathrm{d}y}{\mathrm{d}p}$
 - Sensitivity ODEs: $\frac{\mathrm{d}s}{\mathrm{d}t} = \frac{\partial f}{\partial y}s + \frac{\partial f}{\partial p}$

The final sensitivity ODE expression assumes that there are no intermediate expressions that may implicitly depend on time, parameters or other states.

In myokit, we might also be interested in the sensitivities of an intermediate expression. Those can be straightforwadly derived from the above sensitivities

 - Intermediate: $x = x(y, p, t)$
 - Sensitivities: $\frac{\mathrm{d}x}{\mathrm{d}p} = \frac{\partial x}{\partial y}s + \frac{\partial x}{\partial p}$.

In [7]:
import os

import myokit
from myokit.formats.sbml import SBMLImporter

## Model: Bacterial growth under drug exposure

In this model, we assume that the total bacterial population is composed of a wild type strain $b_w$ and a mutant strain $b_m$
\begin{equation}
    b = b_w + b_m.
\end{equation}

Here $b$, $b_w$ and $b_m$ is the bacterial count. The growth of the wild type under exposure to the drug is modelled by an exponential growth $\lambda $ and a drug-induced killing $\kappa_w $ as well as mutation rate $\mu $
\begin{equation}
    \frac{\text{d}b_w}{\text{d}t} =
        \lambda b_w
        - \kappa _w c b_w
        - \mu c b_w.
\end{equation}

The mutant population is resistant to the drug and experiences a Hill-like killing with maximal killing rate $\kappa _m$ and EC50 $c_{50}$
\begin{equation}
    \frac{\text{d}b_m}{\text{d}t} =
        \lambda b_m
        - \kappa _m\frac{c}{c - c_{50}}b_m
        + \mu c b_w.
\end{equation}

The drug is administered directly to the bacterial environment

\begin{equation}
    \frac{\text{d}a}{\text{d}t} = -ka + r_{\text{dose}}(t), \quad \mathrm{and} \quad c = \frac{a}{v},
\end{equation}

where $a$ is the amount of the drug, $k$ is the elimination rate and $r_{\text{dose}}$ dose rate. $c$ is the drug concentration and $v$ is the effective volume of distribution of the drug.

In [11]:
directory = os.getcwd()
path = os.path.join(directory, 'models/simple_kill_and_resistance_model.xml')
model = SBMLImporter().model(path)
print(model.code())

[[model]]
name: adaptive_resistance
# Initial values
myokit.bacterial_count_adapted     = 1
myokit.bacterial_count_susceptible = 1
central.drug_amount                = 0

[central]
dot(drug_amount) = -(size * myokit.elimination_rate * drug_concentration)
    in [kg (1e-06)]
drug_concentration = drug_amount / size
    in [g/m^3]
size = 1
    in [L]

[myokit]
dot(bacterial_count_adapted) = (growth_rate - kappa_adapted * (central.drug_concentration^gamma / (central.drug_concentration^gamma + concentration_e50_adapted^gamma))) * bacterial_count_adapted + mutation_rate * central.drug_concentration * bacterial_count_susceptible
    in [1/m^3 (1000)]
dot(bacterial_count_susceptible) = (growth_rate - (kappa_susceptible * central.drug_concentration + mutation_rate * central.drug_concentration)) * bacterial_count_susceptible
    in [1/m^3 (1000)]
concentration_e50_adapted = 1
    in [g/m^3]
elimination_rate = 1
    in [S/F (0.0002777777777777778)]
gamma = 1
    in [1]
growth_rate = 1
    in [S/F

## Compute sensitivities analytically

The quantity that's typically measured in experiments is the total bacterial count $b$. So, we are interested in the sensitivities of $b$.

In this case, $b$ does not explicitly depend on any parameters $\frac{\partial b}{\partial p}=0$, and the depends linearly on the states $\frac{\partial b}{\partial b_w} = \frac{\partial b}{\partial b_w} = 1$. Thus, the sensitivities to all parameters are given by
\begin{equation}
    \frac{\mathrm{d}b}{\mathrm{d}p} = 
        \frac{\mathrm{d}b_w}{\mathrm{d}p} + \frac{\mathrm{d}b_m}{\mathrm{d}p}.
\end{equation}

#### Sensitivity to $\lambda$:
For ease of notation we use a superscript to indicate the subgroup and a subscript to indicate the parameter
\begin{equation}
    \frac{\mathrm{d}s^{w}_{\lambda}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{\lambda} + b_w
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{\lambda}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{\lambda} + b_m.
\end{equation}

#### Sensitivity to $\kappa _w$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{\kappa _w}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{\kappa _w} + c b_w
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{\kappa _w}}{\mathrm{d}t} = 
        0.
\end{equation}

#### Sensitivity to $\mu$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{\mu}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{\mu} + c b_w
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{\mu}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{\mu} + c b_m.
\end{equation}

#### Sensitivity to $\kappa _m$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{\kappa _m}}{\mathrm{d}t} = 
        0
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{\kappa _m}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{\kappa _m} + \frac{c}{c - c_{50}}b_w.
\end{equation}

#### Sensitivity to $c_{50}$:
\begin{equation}
    \frac{\mathrm{d}s^{w}_{c_{50}}}{\mathrm{d}t} = 
        0
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{c_{50}}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{c_{50}} + \kappa_m\frac{c}{(c - c_{50})^2}b_w.
\end{equation}

#### Sensitivity to $v$:
The sensitivity to $v$ is interesting, because $b$ depends on $v$ implicitly through either $b_w$ or $b_m$ and $c$. The sensitivity might be abstractly expressed as
\begin{equation}
    \frac{\mathrm{d}s^{i}_v}{\mathrm{d}t} = \frac{\mathrm{d}}{\mathrm{d}v} f(b_i, c, v) =  \frac{\partial f}{\partial b_i}s^{i}_v + \frac{\partial f}{\partial c}\frac{\mathrm{d}c}{\mathrm{d}v} + \frac{\partial f}{\partial v}.
\end{equation}
In this case, $\frac{\partial f}{\partial v} = 0$. Thus, the sensitivities are given by
\begin{equation}
    \frac{\mathrm{d}s^{w}_{v}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _w c - \mu c
        \right) s^{w}_{v} +
        \left( 
            \kappa _w b_w + \mu b_w
        \right) \frac{a}{v^2}
    \quad \mathrm{and} \quad
    \frac{\mathrm{d}s^{m}_{v}}{\mathrm{d}t} = 
        \left( 
            \lambda - \kappa _m\frac{c}{c - c_{50}}
        \right) s^{m}_{v} -
        \left( 
            \mu b_w - \kappa _m\frac{1}{c - c_{50}} b_m + \kappa _m\frac{c}{(c - c_{50})^2} b_m
        \right) \frac{a}{v^2}
\end{equation}